# Homework 2B: Food Safety (Continued)

## Cleaning and Exploring Data with `pandas`

## Due Date: Thursday, Sep 14, 11:59 PM
You must submit this assignment to Gradescope by the on-time deadline, Thursday, Sep 14, 11:59 PM. Please read the syllabus for the grace period policy. No late submissions beyond the grace period will be accepted. **We strongly encourage you to plan to submit your work to Gradescope several hours before the stated deadline.** This way, you will have ample time to reach out to staff for support if you encounter difficulties with submission. While course staff is happy to help guide you with submitting your assignment ahead of the deadline, we will not respond to last-minute requests for assistance (TAs need to sleep, after all!).

Please read the instructions carefully to submit your work to both the coding and written portals of Gradescope.

## Collaboration Policy

Data science is a collaborative activity. While you may talk with others about the homework, we ask that you **write your solutions individually**. If you do discuss the assignments with others, please **include their names** at the top of your notebook.

**Collaborators**: *list collaborators here*


## This Assignment

In this homework, we will continue our exploration of restaurant food safety scores for restaurants in San Francisco. The main goal for this assignment is to focus more on the analysis of the dataset, building on the data cleaning we have done earlier in HW 2A. 


After this homework, you should be comfortable with:
* Reading `pandas` documentation and using `pandas` methods,
* Working with data at different levels of granularity,
* Using `groupby` with different aggregation functions,
* Chaining different `pandas` functions and methods to find answers to exploratory questions.


## Score Breakdown 
Question | Manual | Points
--- | --- | ---
1a | no | 2
1b | no | 3
1c | no | 3
2a | no | 2
2b | no | 3
2c | no | 1
2d | no | 1
3a | no | 1
3b | no | 2
3c | yes | 3
3d | no | 3
3e | yes | 3
4 | no | 3
Total | 2 | 30


## Before You Start

For each question in the assignment, please write down your answer in the answer cell(s) right below the question. 

We understand that it is helpful to have extra cells breaking down the process towards reaching your final answer. If you happen to create new cells below your answer to run code, **NEVER** add cells between a question cell and the answer cell below it. It will cause errors when we run the autograder, and it will sometimes cause a failure to generate the PDF file.

**Important note: The local autograder tests will not be comprehensive. You can pass the automated tests in your notebook but still fail tests in the autograder.** Please be sure to check your results carefully.

Finally, unless we state otherwise, **do not use for loops or list comprehensions**. The majority of this assignment can be done using built-in commands in `pandas` and `NumPy`.  Our autograder isn't smart enough to check, but you're depriving yourself of key learning objectives if you write loops / comprehensions, and you also won't be ready for the midterm.


In [131]:
import numpy as np
import pandas as pd
from IPython.display import display

In HW 2A, we took you through the entire process of reading data from a file to perform some exploration of the data. Here, we again load the dataset that we will be using in HW 2B along with some of the columns we had added in HW 2A. For any additional context regarding the dataset, feel free to revisit HW 2A.

Data cleaning: 
* set encoding to 'ISO-8859-1' (instead of UTF 8)
* rename column business id to bid
* create column postal 5 to contain the 1st 5 character in postal code

For ins
* set timestamp column to date with format format='%m/%d/%Y %I:%M:%S %p'
* column bid = column iid but split by _
* column above -> int

In [132]:
bus = pd.read_csv('data/bus.csv', encoding='ISO-8859-1').rename(columns={"business id column": "bid"})
bus['postal5'] = bus['postal_code'].str[:5]
display(bus.head(5))

ins = pd.read_csv('data/ins.csv')
ins['timestamp'] = pd.to_datetime(ins['date'], format='%m/%d/%Y %I:%M:%S %p')
ins['bid'] = ins['iid'].str.split("_", expand = True)[0].astype(int) 
# Without [0] you'll have both parts of the split strings. the thing is were applying it on 1 col only (bid) -> error
# -> thus must select one only (aka the 1st column [0])
ins.head(5)

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number,postal5
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999,94110
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284,94133
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839,94103
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315,94124
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675,94124


,iid,date,score,type,timestamp,bid
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,2019-03-29,100010
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,2019-04-03,100010
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership,2019-04-17,100017
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,2019-08-16,100017
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup,2019-08-26,100017


<br/><br/>

---


# Question 1: Inspecting the Inspections


## Question 1a

Let's start by looking again at the first 5 rows of `ins` to see what we're working with.

In [133]:
ins.head(5)

,iid,date,score,type,timestamp,bid
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,2019-03-29,100010
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,2019-04-03,100010
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership,2019-04-17,100017
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,2019-08-16,100017
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup,2019-08-26,100017


To better understand how the scores have been allocated, examine how the maximum score varies for each type of inspection. Create a `DataFrame` object `ins_score_by_type`, indexed by all the inspection types (e.g., New Construction, Routine - Unscheduled, etc.), with a single column named `max_score` containing the highest score received. You may find `pd.rename()` to be useful!

In [134]:
ins_score_by_type = ins.groupby('type')['score'].max().to_frame()
ins_score_by_type = ins_score_by_type.rename(columns = {'score': 'max_score'}) 
ins_score_by_type

,max_score
type,
Administrative or Document Review,-1
Community Health Assessment,-1
Complaint,-1
Complaint Reinspection/Followup,-1
Foodborne Illness Investigation,-1
Multi-agency Investigation,-1
New Construction,-1
New Ownership,-1
New Ownership - Followup,-1


<br/>

---

## Question 1b


Given the variability of `ins['score']` observed in 1.a, let's examine the inspection scores `ins['score']` further.

In [135]:
ins['score'].value_counts().head()

score
-1      12632
 100     1993
 96      1681
 92      1260
 94      1250
Name: count, dtype: int64

There are a large number of inspections with a `score` of `-1`. These are probably missing values. Let's see what types of inspections have scores and which do not (score of -1).  We have defined for you a new column `'Missing Score'` that shows `True` if the score for that business is `-1` to help you out with the analysis. 

Use `.groupby` to find out the number of scores that every combination of `type` and `Missing Score` can take on. The result should be a **`DataFrame`** that should look **exactly** as shown below:

In [136]:
ins['Missing score'] = ins['score'] == -1
ins_groups = ins.groupby('type')['Missing score'].value_counts().to_frame()
ins_groups

,,count
type,Missing score,
Administrative or Document Review,True,4
Community Health Assessment,True,1
Complaint,True,1458
Complaint Reinspection/Followup,True,227
Foodborne Illness Investigation,True,115
Multi-agency Investigation,True,3
New Construction,True,994
New Ownership,True,1592
New Ownership - Followup,True,499


Only routine - unscheduled has missing values.

<br/>

---

## Question 1c


Using `groupby` to perform the above analysis gave us a `DataFrame` that wasn't the most readable at first glance. There are better ways to represent the above information that take advantage of the fact that we are looking at combinations of two variables. It's time to pivot (pun intended)!

Create the following `DataFrame`, and assign it to to the variable `ins_missing_score_pivot`. You'll want to use the `pivot_table` method of the `DataFrame` class, which you can read about in the pivot_table [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html). Once you create `ins_missing_score_pivot`, add another column titled `'Total'`, which contains the total number of inspections of that `type`. Sort the table by descending order of `'Total'`.

**Hint:** Consider what happens if there are no values that correspond to a particular combination of `'Missing Score'` and `'type'`. Looking at the documentation for `pivot_table`, is there any function argument that allows you to specify what value to fill in?

If you've done everything right, you should observe that inspection scores appear only to be assigned to `Routine - Unscheduled` inspections and that `ins_missing_score_pivot` looks exactly like below:


<table border="1" class="dataframe" >  <thead>    
    <tr style="text-align: right;">      <th>Missing Score</th>      <th>False</th>      <th>True</th>      <th>Total</th>    </tr>    <tr align="right">      <th>type</th>      <th></th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    
    <tr  align="right">      <th>Routine - Unscheduled</th>      <td>14031</td>      <td>46</td>      <td>14077</td>    </tr>    
    <tr  align="right">      <th>Reinspection/Followup</th>      <td>0</td>      <td>6439</td>      <td>6439</td>    </tr>    
    <tr  align="right">      <th>New Ownership</th>      <td>0</td>      <td>1592</td>      <td>1592</td>    </tr>    
    <tr  align="right">      <th>Complaint</th>      <td>0</td>      <td>1458</td>      <td>1458</td>    </tr>    
    <tr  align="right">      <th>New Construction</th>      <td>0</td>      <td>994</td>      <td>994</td>    </tr>    
    <tr  align="right">      <th>Non-inspection site visit</th>      <td>0</td>      <td>811</td>      <td>811</td>    </tr>    
    <tr  align="right">      <th>New Ownership - Followup</th>      <td>0</td>      <td>499</td>      <td>499</td>    </tr>    
    <tr  align="right">      <th>Structural Inspection</th>      <td>0</td>      <td>394</td>      <td>394</td>    </tr>    
    <tr  align="right">      <th>Complaint Reinspection/Followup</th>      <td>0</td>      <td>227</td>      <td>227</td>    </tr>    
    <tr  align="right">      <th>Foodborne Illness Investigation</th>      <td>0</td>      <td>115</td>      <td>115</td>    </tr>    
    <tr  align="right">      <th>Routine - Scheduled</th>      <td>0</td>      <td>46</td>      <td>46</td>    </tr>    
    <tr  align="right">      <th>Administrative or Document Review</th>      <td>0</td>      <td>4</td>      <td>4</td>    </tr>    
    <tr  align="right">      <th>Multi-agency Investigation</th>      <td>0</td>      <td>3</td>      <td>3</td>    </tr>    
    <tr  align="right">      <th>Special Event</th>      <td>0</td>      <td>3</td>      <td>3</td>    </tr>    
    <tr  align="right">      <th>Community Health Assessment</th>      <td>0</td>      <td>1</td>      <td>1</td>    </tr>  </tbody></table>


In [137]:
ins_missing_score_pivot = ins_groups.pivot_table(
    values = 'count',
    index = 'type',
    columns = 'Missing score',
    fill_value = 0)
ins_missing_score_pivot['Total'] = ins_missing_score_pivot.sum(axis = 1) # difficult couldnt think of this
ins_missing_score_pivot = ins_missing_score_pivot.sort_values(by = 'Total', ascending = False)
ins_missing_score_pivot

Missing score,False,True,Total
type,,,
Routine - Unscheduled,14031.0,46.0,14077.0
Reinspection/Followup,0.0,6439.0,6439.0
New Ownership,0.0,1592.0,1592.0
Complaint,0.0,1458.0,1458.0
New Construction,0.0,994.0,994.0
Non-inspection site visit,0.0,811.0,811.0
New Ownership - Followup,0.0,499.0,499.0
Structural Inspection,0.0,394.0,394.0
Complaint Reinspection/Followup,0.0,227.0,227.0


Notice that inspection scores appear only to be assigned to `Routine - Unscheduled` inspections. It is reasonable for inspection types such as `New Ownership` and `Complaint` to have no associated inspection scores, but we might be curious why there are no inspection scores for the `Reinspection/Followup` inspection type.

<br/><br/>

---

# Question 2: Joining Data Across Tables

In this question, we will start to connect data across multiple tables. We will be using the `merge` function. 

In [138]:
ins

,iid,date,score,type,timestamp,bid,Missing score
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,2019-03-29,100010,True
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,2019-04-03,100010,False
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership,2019-04-17,100017,True
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,2019-08-16,100017,False
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup,2019-08-26,100017,True
...,...,...,...,...,...,...,...
26658,999_20180924,09/24/2018 12:00:00 AM,-1,Routine - Scheduled,2018-09-24,999,True
26659,999_20181102,11/02/2018 12:00:00 AM,-1,Reinspection/Followup,2018-11-02,999,True
26660,999_20190909,09/09/2019 12:00:00 AM,80,Routine - Unscheduled,2019-09-09,999,False
26661,99_20171207,12/07/2017 12:00:00 AM,82,Routine - Unscheduled,2017-12-07,99,False


<br/>

--- 

## Question 2a

Let's figure out which restaurants had the lowest scores. Before we proceed, let's filter out missing scores from `ins` so that negative scores don't influence our results. 

Note that there might be something interesting we could learn from businesses with missing scores, but we are omitting such analysis from this homework. You might consider exploring this for the optional question at the end. 

Note: We have no idea if there is actually anything interesting to learn as we have not attempted this ourselves.

In [139]:
ins = ins[ins['score'] > 0]
ins

,iid,date,score,type,timestamp,bid,Missing score
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,2019-04-03,100010,False
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,2019-08-16,100017,False
15,100041_20190520,05/20/2019 12:00:00 AM,83,Routine - Unscheduled,2019-05-20,100041,False
20,100055_20190425,04/25/2019 12:00:00 AM,98,Routine - Unscheduled,2019-04-25,100055,False
21,100055_20190912,09/12/2019 12:00:00 AM,82,Routine - Unscheduled,2019-09-12,100055,False
...,...,...,...,...,...,...,...
26654,999_20170714,07/14/2017 12:00:00 AM,77,Routine - Unscheduled,2017-07-14,999,False
26656,999_20180123,01/23/2018 12:00:00 AM,80,Routine - Unscheduled,2018-01-23,999,False
26660,999_20190909,09/09/2019 12:00:00 AM,80,Routine - Unscheduled,2019-09-09,999,False
26661,99_20171207,12/07/2017 12:00:00 AM,82,Routine - Unscheduled,2017-12-07,99,False


In [142]:
bus

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number,postal5
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999,94110
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284,94133
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839,94103
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315,94124
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675,94124
...,...,...,...,...,...,...,...,...,...,...
6248,99948,SUSIECAKES BAKERY,3509 CALIFORNIA ST,San Francisco,CA,94118,-9999.000000,-9999.000000,14150452253,94118
6249,99988,HINODEYA SOMA,303 02nd ST STE 102,San Francisco,CA,94107,-9999.000000,-9999.000000,-9999,94107
6250,99991,TON TON,422 GEARY ST,San Francisco,CA,94102,-9999.000000,-9999.000000,14155531280,94102
6251,99992,URBAN EXPRESS KITCHENS LLC,475 06th ST,San Francisco,CA,94103,-9999.000000,-9999.000000,14150368085,94103


We'll start by creating a new `DataFrame` called `ins_named`. It should be exactly the same as `ins`, except that it should have the name and address of every business, as determined by the `bus` DataFrame. 

**Hint**: Use the `merge` method to join the `ins` DataFrame with the appropriate portion of the `bus` DataFrame. See the official [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) on how to use `merge`. The first few rows of the resulting `DataFrame` you create are shown below:

In [145]:
ins_named = pd.merge(
    left = ins,
    right = bus,
    on = 'bid'
)
ins_named

,iid,date,score,type,timestamp,bid,Missing score,name,address,city,state,postal_code,latitude,longitude,phone_number,postal5
0,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,2019-04-03,100010,False,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284,94133
1,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,2019-08-16,100017,False,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839,94103
2,100041_20190520,05/20/2019 12:00:00 AM,83,Routine - Unscheduled,2019-05-20,100041,False,UNCLE LEE CAFE,3608 BALBOA ST,San Francisco,CA,94121,-9999.000000,-9999.000000,-9999,94121
3,100055_20190425,04/25/2019 12:00:00 AM,98,Routine - Unscheduled,2019-04-25,100055,False,Twirl and Dip,335 Martin Luther King Jr. Dr,San Francisco,CA,94118,-9999.000000,-9999.000000,14155300260,94118
4,100055_20190912,09/12/2019 12:00:00 AM,82,Routine - Unscheduled,2019-09-12,100055,False,Twirl and Dip,335 Martin Luther King Jr. Dr,San Francisco,CA,94118,-9999.000000,-9999.000000,14155300260,94118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14026,999_20170714,07/14/2017 12:00:00 AM,77,Routine - Unscheduled,2017-07-14,999,False,SERRANO'S PIZZA II,3274 21st St,San Francisco,CA,94110,37.756997,-122.420534,14155691615,94110
14027,999_20180123,01/23/2018 12:00:00 AM,80,Routine - Unscheduled,2018-01-23,999,False,SERRANO'S PIZZA II,3274 21st St,San Francisco,CA,94110,37.756997,-122.420534,14155691615,94110
14028,999_20190909,09/09/2019 12:00:00 AM,80,Routine - Unscheduled,2019-09-09,999,False,SERRANO'S PIZZA II,3274 21st St,San Francisco,CA,94110,37.756997,-122.420534,14155691615,94110
14029,99_20171207,12/07/2017 12:00:00 AM,82,Routine - Unscheduled,2017-12-07,99,False,J & M A-1 CAFE RESTAURANT LLC,779 Clay St,San Francisco,CA,94108,37.794293,-122.405967,-9999,94108
